In [2]:
import requests
import json
import csv
import pandas as pd
import time
import os


In [6]:
def flatten_json(nested_json, prefix=''):
    """Flattens a nested JSON object."""
    flat_json = {}
    for i in nested_json:
        for k, v in nested_json[i].items():
            new_key = prefix + k
            if isinstance(v, dict):
                flat_json.update(flatten_json(v, new_key + '_'))
            elif isinstance(v, list):
                for index, item in enumerate(v):
                    if isinstance(item,dict):
                        flat_json.update(flatten_json(item, new_key + f'_{index}_'))
                    else:
                        flat_json[new_key + f'_{index}'] = item
            else:
                flat_json[new_key] = v
    return flat_json

def update_csv_from_api(api_url, csv_filename):
    """Fetches data from API, flattens it, and updates CSV."""
    try:
        response = requests.get(api_url,verify=False)
        response.raise_for_status()
        data = response.json()

        if not data:
            print(f"No data received from API: {api_url}. CSV not updated.")
            return

        all_flat_data = []
        for item in data:
            flat_data = flatten_json({item : data[item]})
            all_flat_data.append(flat_data)

        df = pd.DataFrame(all_flat_data)
        df.to_csv(csv_filename, index=False, encoding='utf-8')
        print(f"CSV file '{csv_filename}' updated successfully from {api_url}.")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from API {api_url}: {e}")
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON response from {api_url}: {e}. Response text: {response.text}")
    except Exception as e:
        print(f"An unexpected error occurred processing {api_url}: {e}")

def update_all_csvs(api_urls, base_filename="output"):
    """Updates CSV files for multiple APIs."""
    for i, api_url in enumerate(api_urls):
      csv_filename = f"{base_filename}_{i+1}.csv" # Generates unique filenames
      update_csv_from_api(api_url, csv_filename)

def scheduled_update(api_urls, base_filename="output", interval_seconds=3600):
    """Updates CSV files for multiple APIs at specified intervals."""
    while True:
        update_all_csvs(api_urls, base_filename)
        time.sleep(interval_seconds)

# Example Usage
api_urls = [
    "https://jsonkeeper.com/b/LLQT",
    "https://api.jsonserve.com/rJvd7g",
    "https://api.jsonserve.com/XgAgFJ"
]

base_filename = "data" # base name of the file
# First time update for all APIs
update_all_csvs(api_urls, base_filename)

# To schedule automatic updates for all APIs (e.g., every hour):
# scheduled_update(api_urls, base_filename, 3600) #Uncomment for schedule update

#Example of reading the csv files
dfs = []
for i in range(len(api_urls)):
    try:
        df = pd.read_csv(f"{base_filename}_{i+1}.csv")
        dfs.append(df)
        print(f"Dataframe {i+1} head:\n",df.head())
    except FileNotFoundError:
        print(f"File {base_filename}_{i+1}.csv not found.")
    except pd.errors.EmptyDataError:
        print(f"File {base_filename}_{i+1}.csv is empty.")
    except Exception as e:
        print(f"An error occurred while reading {base_filename}_{i+1}.csv: {e}")

C:\Users\DELL\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'jsonkeeper.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


An unexpected error occurred processing https://jsonkeeper.com/b/LLQT: 'int' object has no attribute 'items'


C:\Users\DELL\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.jsonserve.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


An unexpected error occurred processing https://api.jsonserve.com/rJvd7g: 'int' object has no attribute 'items'


C:\Users\DELL\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.jsonserve.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


An unexpected error occurred processing https://api.jsonserve.com/XgAgFJ: list indices must be integers or slices, not dict
File data_1.csv not found.
File data_2.csv not found.
File data_3.csv not found.


In [8]:
import requests
import json
import csv
import pandas as pd


def flatten_json(nested_json, prefix=''):
    """Flattens a nested JSON object, handling specific structure."""
    flat_json = {}
    for key, value in nested_json.items():
        new_key = prefix + key
        if isinstance(value, dict):
            flat_json.update(flatten_json(value, new_key + '_'))
        elif isinstance(value, list):
            for index, item in enumerate(value):
                if isinstance(item, dict):
                    flat_json.update(flatten_json(item, new_key + f'_{index}_'))
                else:
                    flat_json[new_key + f'_{index}'] = item
        else:
            flat_json[new_key] = value
    return flat_json

def update_csv_from_api(api_url, csv_filename):
    """Fetches data from API, flattens it, and updates CSV."""
    try:
        response = requests.get(api_url, verify=False)
        response.raise_for_status()
        data = response.json()

        if not data:
            print(f"No data received from API: {api_url}. CSV not updated.")
            return

        # Handle specific structure for jsonkeeper.com/b/LLQT
        all_flat_data = []
        quiz_data = data.get('quiz')  # Assuming the key for quiz data is 'quiz'
        if quiz_data:
            # Flatten nested questions (0 to 127)
            questions_0_to_127 = quiz_data.get('questions')
            if questions_0_to_127:
                for index, question in enumerate(questions_0_to_127):
                    flat_question = flatten_json({'question': question})
                    flat_question['question_index'] = index  # Add index for clarity
                    all_flat_data.append(flat_question)

            # Flatten the nested question (index 128)
            question_128 = quiz_data.get('question128')
            if question_128:
                flat_question_128 = flatten_json({'question128': question_128})
                all_flat_data.append(flat_question_128)

        df = pd.DataFrame(all_flat_data)
        df.to_csv(csv_filename, index=False, encoding='utf-8')
        print(f"CSV file '{csv_filename}' updated successfully from {api_url}.")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from API {api_url}: {e}")
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON response from {api_url}: {e}. Response text: {response.text}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# Example usage (replace with your actual API URL)
api_url = "https://jsonkeeper.com/b/LLQT"
csv_filename = "quiz_data.csv"
update_csv_from_api(api_url, csv_filename)

C:\Users\DELL\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'jsonkeeper.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


CSV file 'quiz_data.csv' updated successfully from https://jsonkeeper.com/b/LLQT.


In [10]:
def flatten_json(nested_json, prefix=''):
    """Flattens a nested JSON object, handling nested objects within an 'id' key."""
    flat_json = {}
    for key, value in nested_json.items():
        new_key = prefix + key
        if isinstance(value, dict):
            if key == 'id':  # Handle nested objects within 'id'
                for inner_key, inner_value in value.items():
                    flat_json.update(flatten_json(inner_value, new_key + '_' + inner_key + '_'))
            else:
                flat_json.update(flatten_json(value, new_key + '_'))
        elif isinstance(value, list):
            for index, item in enumerate(value):
                if isinstance(item, dict):
                    flat_json.update(flatten_json(item, new_key + f'_{index}_'))
                else:
                    flat_json[new_key + f'_{index}'] = item
        else:
            flat_json[new_key] = value
    return flat_json

def update_csv_from_api(api_url, csv_filename):
    """Fetches data from API, flattens it, and updates CSV."""
    try:
        response = requests.get(api_url)
        response.raise_for_status()
        data = response.json()

        if not data:
            print(f"No data received from API: {api_url}. CSV not updated.")
            return

        # Handle specific structure for api.jsonserve.com/rJvd7g
        all_flat_data = []
        flat_data = flatten_json(data)
        all_flat_data.append(flat_data)

        df = pd.DataFrame(all_flat_data)
        df.to_csv(csv_filename, index=False, encoding='utf-8')
        print(f"CSV file '{csv_filename}' updated successfully from {api_url}.")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from API {api_url}: {e}")
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON response from {api_url}: {e}. Response text: {response.text}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# Example usage (replace with your actual API URL)
api_url = "https://api.jsonserve.com/rJvd7g"
csv_filename = "data_from_rJvd7g.csv"
update_csv_from_api(api_url, csv_filename)

CSV file 'data_from_rJvd7g.csv' updated successfully from https://api.jsonserve.com/rJvd7g.


In [12]:
def flatten_json(nested_json, prefix=''):
    """Flattens a nested JSON object, handling multiple nested objects."""
    flat_json = {}
    for key, value in nested_json.items():
        new_key = prefix + key
        if isinstance(value, dict):
            flat_json.update(flatten_json(value, new_key + '_'))
        elif isinstance(value, list):
            for index, item in enumerate(value):
                if isinstance(item, dict):
                    flat_json.update(flatten_json(item, new_key + f'_{index}_'))
                else:
                    flat_json[new_key + f'_{index}'] = item
        else:
            flat_json[new_key] = value
    return flat_json

def update_csv_from_api(api_url, csv_filename):
    """Fetches data from API, flattens it, and updates CSV."""
    try:
        response = requests.get(api_url)
        response.raise_for_status()
        data = response.json()

        if not data:
            print(f"No data received from API: {api_url}. CSV not updated.")
            return

        # Handle specific structure for https://api.jsonserve.com/XgAgFJ
        all_flat_data = []
        for item in data:
            flat_data = flatten_json(item)
            all_flat_data.append(flat_data)

        df = pd.DataFrame(all_flat_data)
        df.to_csv(csv_filename, index=False, encoding='utf-8')
        print(f"CSV file '{csv_filename}' updated successfully from {api_url}.")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from API {api_url}: {e}")
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON response from {api_url}: {e}. Response text: {response.text}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# Example usage (replace with your actual API URL)
api_url = "https://api.jsonserve.com/XgAgFJ"
csv_filename = "data_from_XgAgFJ.csv"
update_csv_from_api(api_url, csv_filename)

CSV file 'data_from_XgAgFJ.csv' updated successfully from https://api.jsonserve.com/XgAgFJ.
